In [28]:
import numpy as np
import qibo
from qibo import Circuit, gates

""" 
Auxiliary function that performs the necessary measurements for shadow tomography on the final state of a 
given quantum circuit and returns the chosen measurements and their outcomes.  
Inputs: 
    - circuit (qibo.Circuit) : a circuit template that prepares the state we wish to perform tomography on.
    - shadow_size (int) : the number of snapshots in the shadow.
    - num_qubits (int) : The number of qubits in the circuit.
     
Outputs: 
    - outcomes: a matrix containing the measurement outcomes. 
    - measurement_basis: a matrix containing the indexes for the Pauli measurement basis chosen 
    (0 = X, 1 = Y, 2 = Z).
    On both matrices the position (i, j) corresponds to the i-th snapshot and the j-th qubit
"""
def perform_shadow_snapshots(circuit, shadow_size, num_qubits):

    # Maps the Pauli measurements to the indeces 0, 1, 2 = X, Y, Z
    Pauli = [gates.X, gates.Y, gates.Z]

    # Measurement basis are chosen at random 
    measurement_basis = np.random.randint(0, 3, size = (shadow_size, num_qubits))
    outcomes = np.zeros((shadow_size, num_qubits))

    for i in range(shadow_size):
        # In each snapshot, we perform a measurement in a randomly chosen basis
        c = circuit.copy()
        c.add(gates.M(*range(num_qubits), basis = [Pauli[measurement_basis[i][j]] for j in range(num_qubits) ]))
        outcomes[i] = c(nshots = 1).samples(binary = True)[0]

    # Returns both the measurement outcomes and the basis the measurements were performed in
    return (outcomes, measurement_basis)

"""
Given a quantum circuit, performs shadow tomography on its final state and returns a classical shadow
of the specified size
"""
def calculate_classical_shadow(circuit, shadow_size, num_qubits):

    # Performs the measurements
    outcomes, measurement_basis = perform_shadow_snapshots(circuit, shadow_size, num_qubits)

    # Possible states in matrix form
    states = [np.array([[1, 0],[0, 0]]), np.array([[0, 0], [0, 1]])]
    # Matrices to invert the implicit unitary operations performed when measuring in different basis
    unitaries = [gates.H(0).matrix(), gates.H(0).matrix()@gates.S(0).dagger().matrix(), gates.I(0).matrix()]
    shadow = np.full(shadow_size, [1], dtype = np.ndarray)

    # Computes the classical shadows for each snapshots
    for i in range(shadow_size):
        for j in range(num_qubits):
            state = states[int(outcomes[i][j])]
            U = unitaries[measurement_basis[i][j]]
            local_rho = 3*(U.conj().T @ state @ U) - np.eye(2)
            shadow[i] = np.kron(shadow[i], local_rho)

    return shadow

"""
Reconstructs a state using shadow tomography (although this is not
 the intended application of the method)
"""
def shadow_state_reconstruction(circuit, shadow_size, num_qubits):
    
    shadow = calculate_classical_shadow(circuit, shadow_size, num_qubits)

    shadow_rho = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(shadow_size):
        shadow_rho = shadow_rho + shadow[i]
    return shadow_rho / shadow_size
       

def estimate_observables(circuit, observables, N, K, num_qubits):
    
    shadow = calculate_classical_shadow(circuit, N, num_qubits)
    M = observables.size
    results = np.empty(M)
    chunk_size = N//K
    chunk_rhos = np.zeros(K, dtype = np.ndarray)
    for k in range(K):
        for j in range (k*chunk_size, (k + 1)*chunk_size):
            chunk_rhos[k] = chunk_rhos[k] + shadow[j]
        chunk_rhos[k] = chunk_rhos[k]/chunk_size
    
    for i in range(M):
        results[i] = np.median([(observables[i]@chunk_rhos[k]).trace() for k in range(K)])

    return results

def shadow_norm(obs):
    num_qubits = int(np.log2(obs.shape[0]))
    return np.linalg.norm(obs - np.trace(obs)/(2 ** num_qubits)*np.eye(2**n), ord = np.inf)

def shadow_bound(error, observables, failure_rate = 0.01):

    M = observables.size
    K = 2 * np.log(2 * M / failure_rate)
    N = 34 * max(shadow_norm(o)**2 for o in observables) / error**2
    return int(np.ceil(N * K)), int(K)

n = 2 #number of qubits
m = 1
M = 4
rng = np.random.default_rng()
theta = rng.uniform(0, np.pi, 2*n*m)
Pauli = [gates.I(0).matrix(), gates.X(0).matrix(), gates.Y(0).matrix(), gates.Z(0).matrix()]
c = Circuit(n, density_matrix = True)

for i in range(m): 
    rho = [1]
    for j in range(n):
        c.add(gates.RY(j, theta[2*n*i+2*j]))
        c.add(gates.RZ(j, theta[2*n*i+2*j+1]))
        c.add(gates.CNOT(j, (j+1)%n))


rho_shadow = shadow_state_reconstruction(c, 10000, n)
print(((rho_shadow)@c().state()).trace())

# observables = np.full(M, [1], dtype = np.ndarray)

# for m in range(M):
#     for i in range(n):
#         observables[m] = np.kron(observables[m], Pauli[np.random.randint(4)])
    
# print("Real values:")
# for m in range(M):
#     print((observables[m]@c.copy()().state()).trace())

# print()
# print("Shadow approximations:")
# N, K = shadow_bound(0.1, observables)
# print(N, K)
# shadow_results = estimate_observables(c, observables, N, K, n)
# for m in range(M):
#     print(shadow_results[m])






(1.0125409902917364+8.05020113070487e-18j)
